# Getting Started with Landsat Surface Reflectance

This example demonstrates the basic workflow for accessing and working with Landsat surface reflectance data in the Open Data Cube. We define a small study area, load relevant spectral bands for a selected time period, and visualise the data both as static images and in an interactive map. Finally, we export the processed imagery as a Cloud Optimised GeoTIFF for external use.  

## Import libraries

We begin by importing the Python libraries required to connect to the datacube, configure data access, define our study area, and apply Landsat-specific utilities.  

In [ ]:
#Import required libraries and packages
import os

from datacube import Datacube
from datacube.utils.aws import configure_s3_access
from odc.geo.geom import point
from piksel.utils import patch_usgs_landsat

## Configure the environment

We configure AWS S3 access to ensure the notebook can retrieve Landsat surface reflectance data from cloud storage, and create a `Datacube` instance for querying and loading datasets.  

In [ ]:
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

if "AWS_NO_SIGN_REQUEST" in os.environ:
    del os.environ["AWS_NO_SIGN_REQUEST"]

configure_s3_access(requester_pays=True)

dc = Datacube()

## Pick a study area

We define a location of interest using latitude and longitude coordinates, create a small bounding box around this point, and set display parameters for visualisation.

In [ ]:
# Find a location you're interested in on Google Maps and copy the coordinates
# by right-clicking on the map and clicking the coordinates

# These coords are in the order Y then X, or Latitude then Longitude
coords = -8.65, 115.20  # Denpasar, Bali
aoi_point = point(coords[1], coords[0], crs="EPSG:4326")
bbox = aoi_point.buffer(0.2).boundingbox

landsat_stretch = dict(vmin=7500, vmax=12000)

datetime = "2025-05"

# Preview the area
bbox.explore(zoom=8)

## Load data

This uses the Datacube library to handle loading of the actual data. The `dask_chunks` argument instructs the tool to use Dask
to lazy-load the data.

In [ ]:
# Load Landsat 9 Collection 2 Level 2 Surface Reflectance data for the selected area and time range

data = dc.load(
    product="ls9_c2l2_sr",
    measurements=["red", "green", "blue"],
    output_crs="EPSG:32750",
    resolution=30,
    time=datetime,
    longitude=(bbox.left, bbox.right),
    latitude=(bbox.bottom, bbox.top),
    dask_chunks={"time": 1, "x": 512, "y": 512},
    group_by="solar_day"
)

data

## Visualise data

This step uses `matplotlib` to view data as a static image. It takes a longer time to
run than previous steps, because it's actually loading the data to prepare the images.

Note the xarray dataset we loaded in the above cell has 4 images (We can tell from the `time dimension`). Now let's plot all four images at once.

The `to_array()` function is used to convert the data from an xarray dataset to an array format to allow us to visualise the data as a
red, green, blue "true colour" image.

In [ ]:
data.to_array().plot.imshow(col="time", col_wrap=2, size=6, **landsat_stretch)

### Interactive map

This step uses another `odc` tool to visualise the data on a map.

Based on the four images plotted above, we determined that the image captured on 2025-05-04 is the most suitable for our analysis. We can select this image from the xarray.Dataset data using the .sel function, specifying the time parameter as "2025-05-04".

In [ ]:
best = data.sel(time="2025-05-04").squeeze()

# The odc.explore function will choose red, green and blue by default
# but you can choose bands with the bands=["b1", "b2", "b3"] argument
best.odc.explore(**landsat_stretch)

## Export data

Here we write data to disk, again using an `odc` tool.

In [ ]:
rgba = best.odc.to_rgba(**landsat_stretch)
rgba.odc.write_cog("landsat_sr_example.tif", overwrite=True)